In [1]:
import random
import numpy as np
import importlib
import matplotlib.pyplot as plt
from keras.utils import to_categorical
import os
os.chdir('C:/Users/Pierre Lavigne/Dropbox/Etude de Cas M2/Scripts') #Select your working directory
cwd = os.getcwd()
F=importlib.import_module("Functions")
F=importlib.reload(F)


(X_train, train_label) = F.load_pickled_data("C:/Users/Pierre Lavigne/Desktop/data/train.p", ['features', 'labels'])
(X_test, test_label) = F.load_pickled_data("C:/Users/Pierre Lavigne/Desktop/data/test.p", ['features', 'labels'])
X_train = X_train/255.
X_test = X_test/255.

x_train = X_train[:1000]
train_label = train_label[:1000]

x_test = X_test[test_label == 41]
x_test = np.append(x_test,X_test[test_label == 31],axis=0)
x_test = np.append(x_test,X_test[test_label == 36],axis=0)

y_test = test_label[test_label == 41]
y_test = np.append(y_test,test_label[test_label == 31],axis =0)
y_test = np.append(y_test,test_label[test_label == 36],axis =0)

(x_train,train_label) = F.increase_data(x_train,10,train_label)
(x_test,test_label) = F.increase_data(x_test,10,y_test)


(x_train,train_label) = F.flip_extend(x_train,train_label)
(x_test,test_label) = F.flip_extend(x_test,test_label)

from skimage.util import random_noise

x_train_noisy = np.array([random_noise(x, mode='pepper', seed=None, clip=True) for x in x_train])
x_train=F.preprocess_dataset_new(x_train)
x_train_noisy=F.preprocess_dataset_new(x_train_noisy)

x_test_noisy = np.array([random_noise(x, mode='pepper', seed=None, clip=True) for x in x_test])
x_test=F.preprocess_dataset_new(x_test)
x_test_noisy=F.preprocess_dataset_new(x_test_noisy)


x_test = x_test.astype('float32') 
x_train = x_train.astype('float32') 
x_test_noisy = x_test_noisy.astype('float32') 
x_train_noisy = x_train_noisy.astype('float32') 


x_train = np.reshape(x_train, (len(x_train), 32, 32, 1))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 32, 32, 1))  # adapt this if using `channels_first` image data format
x_train_noisy = np.reshape(x_train_noisy, (len(x_train), 32, 32, 1))  # adapt this if using `channels_first` image data format
x_test_noisy = np.reshape(x_test_noisy, (len(x_test), 32, 32, 1))  # adapt this if using `channels_first` image data format

x_test = np.append(x_test,x_test_noisy,axis = 0)
x_train = np.append(x_train,x_train_noisy,axis = 0)
train_label = np.append(train_label,train_label,axis = 0)
test_label = np.append(test_label,test_label,axis = 0)

from keras.utils import to_categorical
target_test=to_categorical(test_label)
target_train=to_categorical(train_label)

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from keras.callbacks import EarlyStopping

early_stopping_monitor = EarlyStopping(patience=2)

input_shape=x_train[1].shape
                   
model = Sequential()
model.add(Conv2D(16, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(target_train.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer="adam",
              metrics=['accuracy'])


model.fit(x_train,target_train,
          epochs=20,
          validation_data=(x_test, target_test),
          callbacks=[early_stopping_monitor])

model.evaluate(x_test,target_test)

Using TensorFlow backend.


0.0 % de progression
10.0 % de progression
20.0 % de progression
30.0 % de progression
40.0 % de progression
50.0 % de progression
60.0 % de progression
70.0 % de progression
80.0 % de progression
90.0 % de progression
0.0 % de progression
22.22222222222222 % de progression
44.44444444444444 % de progression
66.66666666666666 % de progression
88.88888888888889 % de progression


C:\Users\Pierre Lavigne\Anaconda3\lib\site-packages\skimage\util\dtype.py:110: UserWarning: Possible precision loss when converting from float64 to uint16
  "%s to %s" % (dtypeobj_in, dtypeobj))


Train on 46200 samples, validate on 23940 samples
Epoch 1/20
46200/46200 [==============================] - 142s 3ms/step - loss: 0.1724 - acc: 0.9482 - val_loss: 0.1363 - val_acc: 0.9513
Epoch 2/20
46200/46200 [==============================] - 142s 3ms/step - loss: 0.0519 - acc: 0.9834 - val_loss: 0.2098 - val_acc: 0.9392
Epoch 3/20
23940/23940 [==============================] - 19s 792us/step


[0.1977079807804357, 0.947953216374269]